In [ ]:
import os
import uuid
import hashlib
from urllib.parse import urlencode
import requests
from pprint import pprint
import jwt


In [ ]:
ACCESS_KEY = 'your access key'
SECRET_KEY = 'your secret key'
SERVER_URL = 'https://api.upbit.com'

In [ ]:
def getAskPrice(coin):
    queryString = {"markets":"KRW-"+coin}
    headers = {"Accept": "application/json"}

    response = requests.request("GET", SERVER_URL+'/v1/orderbook', headers=headers, params=queryString)

    HTTP_OK = 200
    if response.status_code == HTTP_OK:
        pprint(response.json())
        orderBookUnits = response.json()[0]['orderbook_units']
        askPrice = orderBookUnits[0]['ask_price']
        return float(askPrice)
    else :
        return -1

In [ ]:
price = getAskPrice('BTC')
print(price)

In [ ]:
def getCoinAmount(coin):
    payload = {
        'access_key': ACCESS_KEY,
        'nonce': str(uuid.uuid4()),
    }

    jwt_token = jwt.encode(payload, SECRET_KEY)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.get(SERVER_URL + "/v1/accounts", headers=headers)
    
    for asset in res.json():
        if asset['currency'] == coin:
            return float(asset['balance'])

    return -1

In [ ]:
getCoinAmount('TRX')

In [ ]:
def bid(coin, price) :
    query = {
        'market': 'KRW-' + coin,
        'side': 'bid',  # buy
        #'volume': '0.01',
        'price': str(price),
        'ord_type': 'price', # buy
    }
    query_string = urlencode(query).encode()

    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {
        'access_key': ACCESS_KEY,
        'nonce': str(uuid.uuid4()),
        'query_hash': query_hash,
        'query_hash_alg': 'SHA512',
    }

    jwt_token = jwt.encode(payload, SECRET_KEY)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}

    res = requests.post(SERVER_URL+'/v1/orders', params=query, headers=headers)
    print(res)

In [ ]:
bid('ETH', 20000)

In [ ]:
def ask(coin, volume) :
    query = {
        'market': 'KRW-' + coin,
        'side': 'ask',  # sell
        'volume': str(volume),
        #'price': '100000',
        'ord_type': 'market', # sell
    }
    query_string = urlencode(query).encode()

    m = hashlib.sha512()
    m.update(query_string)
    query_hash = m.hexdigest()

    payload = {
        'access_key': ACCESS_KEY,
        'nonce': str(uuid.uuid4()),
        'query_hash': query_hash,
        'query_hash_alg': 'SHA512',
    }

    jwt_token = jwt.encode(payload, SECRET_KEY)
    authorize_token = 'Bearer {}'.format(jwt_token)
    headers = {"Authorization": authorize_token}
    
    res = requests.post(SERVER_URL+'/v1/orders', params=query, headers=headers)
    print(res)

In [ ]:
volume = getCoinAmount('ETH')
ask('ETH', volume)